In [2]:
from unified_planning.io.pddl_reader import PDDLReader
from unified_planning.model import Problem
from unified_planning.shortcuts import *
from bitblast.helpers.utils import *
from pathlib import Path
import os
import numpy as np
from unified_planning.io.pddl_writer import PDDLWriter
from unified_planning.model.walkers import LinearChecker, FreeVarsExtractor, Substituter, Simplifier, Nnf
from sympy.parsing.sympy_parser import parse_expr, standard_transformations

In [3]:
reader = PDDLReader()

problem = reader.parse_problem(Path("/home/lubonassi/code/BitBlast/test/pddl/counters/domain.pddl"), Path("/home/lubonassi/code/BitBlast/test/pddl/counters/instance_4.pddl"))

In [4]:
problem

problem name = instance_4

types = [counter]

fluents = [
  real value[c=counter]
  real max_int
]

actions = [
  action increment(counter c) {
    preconditions = [
      ((value(c) + 1) <= max_int)
    ]
    effects = [
      value(c) += 1
    ]
  }
  action decrement(counter c) {
    preconditions = [
      (1 <= value(c))
    ]
    effects = [
      value(c) -= 1
    ]
  }
]

objects = [
  counter: [c0, c1, c2, c3]
]

initial fluents default = [
]

initial values = [
  max_int := 8
  value(c0) := 0
  value(c1) := 0
  value(c2) := 0
  value(c3) := 0
]

goals = [
  (((value(c0) + 1) <= value(c1)) and ((value(c1) + 1) <= value(c2)) and ((value(c2) + 1) <= value(c3)))
]


In [5]:
simplifier = Simplifier(environment=get_environment(), problem=problem)
checker = LinearChecker(environment=get_environment(), problem=problem)

In [6]:
# Transform all effects into INCREASE
def set_normalized_effect(act: InstantaneousAction, eff: Effect):
    if eff.is_increase():
        act.add_increase_effect(condition=eff.condition, value=eff.value, fluent=eff.fluent)
    elif eff.is_decrease():
        act.add_increase_effect(condition=eff.condition, value=simplifier.simplify(-eff.value), fluent=eff.fluent)
    else:
        # TODO: handle increase and decrease effects expressed as assignments
        raise NotImplementedError("Effect type not supported")
    

def normalize_effects(act: InstantaneousAction) -> InstantaneousAction:
    normalized_action = InstantaneousAction(act.name)
    for pre in act.preconditions:
        normalized_action.add_precondition(pre)
    
    for eff in act.effects:
        set_normalized_effect(normalized_action, eff)
    return normalized_action
        

In [9]:
normalized_actions = [normalize_effects(act) for act in problem.actions]
action_map = get_action_effect_map(normalized_actions)
action_map

{'increment': {value(c) += 1}, 'decrement': {value(c) += -1}}

In [ ]:
def lhs(formula: FNode):
    return formula.args[0]

def rhs(formula: FNode):
    return formula.args[1]

In [ ]:
numeric_conditions = 

In [ ]:
simplifier.simplify(tmp)

((((value(c) + 1) * 5) - 8) <= 0)